# San Francisco Rental Prices Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions that can be used for a Panel dashboard.

In [ ]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv

In [2]:
# Read the Mapbox API key
load_dotenv()
mapbox_token = os.getenv("mapbox")

# Import Data

In [3]:
# Import the CSVs to Pandas DataFrames
file_path = Path("Data/sfo_neighborhoods_census_data.csv")
sfo_data = pd.read_csv(file_path, index_col="year")

file_path = Path("Data/neighborhoods_coordinates.csv")
df_neighborhood_locations = pd.read_csv(file_path)

- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

In [4]:
#Data
housing_units = sfo_data['housing_units'].groupby('year').mean()
avg_gross_rent = sfo_data['gross_rent'].groupby('year').mean()
avg_price_year = sfo_data['sale_price_sqr_foot'].groupby('year').mean()


# Average Calculation by Year and Neighborhood
df_groupby = sfo_data.groupby(['year', 'neighborhood'])

avg_price = df_groupby['sale_price_sqr_foot'].mean()
avg_units = df_groupby['housing_units'].mean()
avg_rent = df_groupby['gross_rent'].mean()

new_df = pd.DataFrame({
    'avg_price' : avg_price,
    'housing_units': avg_units,
    'gross_rent' : avg_rent,
})

new_df = new_df.reset_index()
avg_price = avg_price.reset_index()


# Most Expensive 

neighborhoods_sort = new_df.groupby('neighborhood').mean()
neighborhoods_sort = neighborhoods_sort.drop(columns='year')

most_expensive = neighborhoods_sort.nlargest(10, 'avg_price').reset_index()

# Map Data

neighborhood_df = sfo_data.groupby('neighborhood').mean().reset_index()
    
combined_df = pd.concat([df_neighborhood_locations, neighborhood_df], axis=1)
    
px.set_mapbox_access_token(mapbox_token)

In [5]:
# Define Panel Visualization Functions
def housing_units_per_year():
    """Housing Units Per Year."""
    
    housing_units_plot = px.bar(
        housing_units,
        title = 'Housing Units in San Fransisco Per Year - from 2010 to 2016',
        range_y =[370000,387500],
    )

    return housing_units_plot

def average_gross_rent():
    """Average Gross Rent in San Francisco Per Year."""

    avg_gross_rent_plot = px.line(
        avg_gross_rent, 
        title = "Average Gross Rent in San Francisco Per Year."
    )
    
    return avg_gross_rent_plot

def average_sales_price():
    """Average Sales Price Per Year."""
    
    avg_price_plot = px.line(
        avg_price_year, 
        title = "Average Sales Price Per Year."
    )

    return avg_price_plot

def average_price_by_neighborhood(neighborhood):
    """Average Prices by Neighborhood."""

    avg_price_n_plot = px.line(
        avg_price[avg_price['neighborhood'] == neighborhood],
        y = 'sale_price_sqr_foot',
        x = 'year',
        color = 'neighborhood',
        title = f'Average Prices by {neighborhood}'
    )

    return avg_price_n_plot

def top_most_expensive_neighborhoods():
    """Top 10 Most Expensive Neighborhoods."""
   
    plot = px.bar(
        most_expensive, 
        x = 'neighborhood', 
        y = 'avg_price',
        color = 'neighborhood',
        title = 'Top 10 Most Expensive Neighborhoods'
        )

    return plot

def parallel_coordinates():
    """Parallel Coordinates Plot."""
    parallel_coordonates = px.parallel_coordinates(
        most_expensive, 
        dimensions = ['avg_price','housing_units', 'gross_rent'], 
        color = 'avg_price'
    )

    return parallel_coordonates

def parallel_categories():
    """Parallel Categories Plot."""
    parallel_categories = px.parallel_categories(
        most_expensive, 
        dimensions = ['neighborhood','avg_price','housing_units', 'gross_rent'], 
        color = 'avg_price'
    )

    return parallel_categories
def neighborhood_map():
    """Neighborhood Map""" 
    neighborhood_plot = px.scatter_mapbox(
        combined_df, 
        lat = 'Lat',
        lon = 'Lon',
        size = 'sale_price_sqr_foot',
        color = 'gross_rent',
        color_continuous_scale = px.colors.cyclical.IceFire,
        title = 'Average Sale Price Per Square Foot adn Gross Rent in San Francisco',
        zoom = 10, 
        width = 1000
     )
    return neighborhood_plot

In [6]:
import panel as pn
from panel.interact import interact
pn.extension()

## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [7]:
yearly_market = pn.Row(
    housing_units_per_year(),
    average_gross_rent(),
    average_sales_price()
)

neighborhood_analysis =pn.Column(
    interact(average_price_by_neighborhood, neighborhood = list (avg_price['neighborhood'])),
    top_most_expensive_neighborhoods()
)

parallel_analysis = pn.Column(
    parallel_coordinates(),
    parallel_categories()
)


In [8]:
panel = pn.Tabs(
    ("Neighborhood Map", neighborhood_map()),
    ('Yearly Market Analysis', yearly_market),
    ('Neighborhood Analysis', neighborhood_analysis),
    ('Parallel Plots Anlaysis', parallel_analysis)
)


## Serve the Panel Dashboard

In [9]:
panel.servable()

Tabs
    [0] Plotly(Figure, name='Neighborhood Map')
    [1] Row
        [0] Plotly(Figure)
        [1] Plotly(Figure)
        [2] Plotly(Figure)
    [2] Column
        [0] Column
            [0] Column
                [0] Select(name='neighborhood', options=['Alamo Square', ...], value='Alamo Square')
            [1] Row
                [0] Plotly(Figure, name='interactive01468')
        [1] Plotly(Figure)
    [3] Column
        [0] Plotly(Figure)
        [1] Plotly(Figure)